In [1]:
import random

import torch
import torch.nn as nn
import numpy as np

from models.UNet_3Plus import UNet_3Plus_DeepSup_CGM, UNet_3Plus_DeepSup
from loss.bceLoss import BCE_loss
from loss.iouLoss import IOU,IOU_loss
from loss.msssimLoss import MSSSIM

from dataloader.cocoform_loader import CustomDataLoader

pytorch version: 1.7.1+cu110
GPU 사용 가능 여부: True
Tesla V100-SXM2-32GB
1


In [2]:
# seed 고정
random_seed = 21
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

In [3]:
batch_size = 4

In [4]:
common_json_path = "/opt/ml/segmentation/input/data/"
train_json_path = common_json_path + "train.json"
val_json_path = common_json_path + "val.json"
test_json_path = common_json_path + "test.json"

In [5]:
# collate_fn needs for batch
def collate_fn(batch):
    return tuple(zip(*batch))

import albumentations as A
from albumentations.pytorch import ToTensorV2

train_transform = A.Compose([
                            ToTensorV2()
                            ])

val_transform = A.Compose([
                          ToTensorV2()
                          ])

test_transform = A.Compose([
                           ToTensorV2()
                           ])

# train dataset
train_dataset = CustomDataLoader(
    common_dir=common_json_path, json_path=train_json_path, 
    mode='train', transform=train_transform
)

# validation dataset
val_dataset = CustomDataLoader(
    common_dir=common_json_path, json_path=val_json_path, 
    mode='val', transform=val_transform
)

# test dataset
test_dataset = CustomDataLoader(
    common_dir=common_json_path, json_path=test_json_path, 
    mode='test', transform=test_transform
)
# DataLoader
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset, 
    batch_size=batch_size,
    shuffle=True,
    num_workers=4,
    collate_fn=collate_fn
)

val_loader = torch.utils.data.DataLoader(
    dataset=val_dataset, 
    batch_size=batch_size,
    shuffle=False,
    num_workers=4,
    collate_fn=collate_fn
)

test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    # num_workers=4,
    collate_fn=collate_fn
)

loading annotations into memory...
Done (t=3.78s)
creating index...
index created!
loading annotations into memory...
Done (t=1.24s)
creating index...
index created!
loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


In [6]:
# UNet_3Plus_DeepSup, UNet_3Plus_DeepSup_CGM
model = UNet_3Plus_DeepSup_CGM(
    in_channels=3,
    n_classes=len(train_dataset.category_names),
    # feature_scale=4,
    is_deconv=True,
    is_batchnorm=True
)
model = model.cuda()

In [7]:
a_batch = next(iter(train_loader))
x, y, cls_label = list(map(torch.stack, a_batch))
cls_branch, Dec = model(x.cuda())

In [8]:
bce_loss = nn.BCELoss().cuda()
bce_out = bce_loss(cls_branch, cls_label.cuda())

In [9]:
iou_loss = IOU(size_average=True)
for D in Dec:
    iou_out = iou_loss(D, y.cuda())

In [10]:
ms_ssim = MSSSIM()
for D in Dec:
    msssim_out = ms_ssim(D, y.cuda())